In [0]:
#@title Import Data
#@markdown - upload the folder aml-task4 into your google drive
#@markdown - change the runtime to gpu (is faster than tpu)
#@markdown - run this cell to import the data (to see the code: right mouse click / form / show code )


# First remember to change runtime to GPU !!

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

#COLAB
!mkdir -p drive
!google-drive-ocamlfuse drive
import os
#Upload on drive fist the data folder called "data"
os.chdir("/content/drive/aml-task4")

In [0]:
#@title Install Packages
#@markdown this cell installs all required packages using pip

!pip install sk-video
!pip install natsort
!apt-get install ffmpeg

In [0]:
import sys
import os
import natsort
import pandas as pd
import numpy as np
import skvideo.io
import tensorflow as tf

from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import train_test_split

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from keras import optimizers
from keras.optimizers import Adam
from sklearn.metrics import roc_auc_score
from keras.utils import np_utils
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint

In [0]:
#@title Provided Helper Functions
def _int64_feature(value):
	return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
	return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def save_tf_record(x,file_name,y = None):
    writer = tf.python_io.TFRecordWriter(file_name)
    if y is None:
        for video in x:
            sys.stdout.flush()
            feature = {'len': _int64_feature(video.shape[0]),
                       'height': _int64_feature(video.shape[1]),
                       'width': _int64_feature(video.shape[2]),
                       'video': _bytes_feature(tf.compat.as_bytes(video.tostring()))}
            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())
    else:
        for video,label in zip(x,y):
            sys.stdout.flush()
            feature = {'len': _int64_feature(video.shape[0]),
                       'height': _int64_feature(video.shape[1]),
                       'width': _int64_feature(video.shape[2]),
                       'video': _bytes_feature(tf.compat.as_bytes(video.tostring())),
                       'label': _int64_feature(label)}
            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())
    
    writer.close()
    sys.stdout.flush()

def prob_positive_class_from_prediction(pred):
    return np.array([p['probabilities'][1] for p in pred])

def decode(serialized_example):
    features = tf.parse_single_example(
        serialized_example,
        features={
            'len': tf.FixedLenFeature([], tf.int64),
            'height': tf.FixedLenFeature([], tf.int64),
            'width': tf.FixedLenFeature([], tf.int64),
            'label': tf.FixedLenFeature([], tf.int64,default_value = 0),
            'video': tf.FixedLenFeature([], tf.string),
        })
    video = tf.decode_raw(features['video'], tf.uint8)
    height = features['height']
    width = features['width']
    length = features['len']
    shape = tf.stack([length,height,width])
    video = tf.reshape(video,shape)
    label = features['label']
    features = {'video':video}
    return features,label

def input_fn_from_dataset(files,batch_size = 1,num_epochs = None,shuffle = True):
    data_set = tf.data.TFRecordDataset(files)
    if shuffle:
        data_set = data_set.shuffle(buffer_size=len(files)) 
    data_set = data_set.map(decode)
    data_set = data_set.padded_batch(batch_size,padded_shapes= ({'video':[212,100,100]},[]))
    data_set = data_set.repeat(num_epochs)
    data_set = data_set.prefetch(batch_size)
    
    return data_set

def decode_frame(serialized_example):
    features = tf.parse_single_example(
        serialized_example,
        features={
            'len': tf.FixedLenFeature([], tf.int64),
            'height': tf.FixedLenFeature([], tf.int64),
            'width': tf.FixedLenFeature([], tf.int64),
            'label': tf.FixedLenFeature([], tf.int64,default_value = 0),
            'video': tf.FixedLenFeature([], tf.string),
        })
    video = tf.decode_raw(features['video'], tf.uint8)
    height = features['height']
    width = features['width']
    length = features['len']
    shape = tf.stack([length,height,length])
    video = tf.reshape(video,shape)
    label = features['label']
    label = tf.expand_dims(label,axis=-1)
    label = tf.tile(label,tf.expand_dims(length,axis=-1))
    features = {'frame':video}
    return features,label

def input_fn_frame_from_dataset(files,batch_size = 1,num_epochs = None):
	data_set = tf.data.TFRecordDataset(files)
	data_set = data_set.shuffle(buffer_size=len(files)) 
	data_set = data_set.map(decode_frame)
	data_set = data_set.apply(tf.contrib.data.unbatch())
	data_set = data_set.shuffle(buffer_size=batch_size)
	data_set = data_set.batch(batch_size)
	data_set = data_set.repeat(num_epochs)
	data_set = data_set.prefetch(batch_size)
	
	return data_set

In [0]:
#@title Data Loading/Saving Functions



def get_videos_from_folder(data_folder):
    '''
    get a list of video x wehre each video is a numpy array in the format [n_frames,width,height] 
    with uint8 elements.
    argument: relative path to the data_folder from the source folder.
    '''
    data_folder = os.path.join(dir_path,data_folder)
    x = []
    file_names = []
    
    if os.path.isdir(data_folder):
        for dirpath, dirnames, filenames in os.walk(data_folder):
            #print(filenames)
            filenames = natsort.natsorted(filenames,reverse=False)
            #filenames.sort(key=lambda f: int(filter(str.isdigit, f)))
            #print(filenames)
            for filename in filenames:
                file_path = os.path.join(dirpath, filename)
                statinfo = os.stat(file_path)
                if statinfo.st_size != 0:
                    video = skvideo.io.vread(file_path, outputdict={"-pix_fmt": "gray"})[:, :, :, 0]
                    x.append(video)
                    file_names.append(int(filename.split(".")[0]))

    indices = sorted(range(len(file_names)), key=file_names.__getitem__)
    x = np.take(x,indices)
    return x

def get_target_from_csv(csv_file):
    '''
    get a numpy array y of labels. the order follows the id of video. 
    argument: relative path to the csv_file from the source folder.
    '''
    csv_file = os.path.join(dir_path,csv_file)
    with open(csv_file, 'r') as csvfile:
        label_reader = pd.read_csv(csvfile)
        y = label_reader['y']
        
    y = np.array(y)
    return y

  
def save_solution(csv_file,prob_positive_class):
    with open(csv_file, 'w') as csv:
        df = pd.DataFrame.from_dict({'id':range(len(prob_positive_class)),'y': prob_positive_class})
        df.to_csv(csv,index = False)
        

def to_frame_based(X, y=None, random_state=None):
  """
  X: array of videos each [n_frames, width, height]
  y: array of video labels
  """ 

  n_videos = X.shape[0]
  
  if y is not None:
    assert(y.shape[0]==n_videos)
    
  # X.shape[1] == n_frames in respective video
  assert(X[0].shape[1]==100)
  assert(X[0].shape[2]==100)
  
  
  frames = []
  frame_labels = []
  idxs = []
  
  for v_id in range(n_videos):
    frames.append(X[v_id])
    
    n_frames = X[v_id].shape[0]
    
    if y is not None:
      labels = np.repeat(y[v_id], n_frames)
      frame_labels.append(labels)
    
    idx = [f"{v_id}_{frame_id}" for frame_id in range(n_frames)]
    idxs.append(idx)
    
  X_frame = np.concatenate(frames)
  idx_frame = np.concatenate(idxs)
  
  s = np.arange(X_frame.shape[0])
  if random_state is not None:
    np.random.RandomState(seed=random_state).shuffle(s)
  
  if y is not None:
    y_frame = np.concatenate(frame_labels)
    return X_frame[s], y_frame[s], idx_frame[s]
  else:
    return X_frame[s], idx_frame[s]


def combine_frame_based_pred(y_pred, idx_frame, reducer_fn):
  assert(y_pred.shape[0]==idx_frame.shape[0])
  
  
  d = {}
  
  for i in range(idx_frame.shape[0]):
    v_id = idx_frame[i].split("_")[0]
    if v_id not in d:
       d[v_id] = []
    d[v_id].append(y_pred[i])
  
  
  predictions = []
  for v_id, preds in d.items():
  
    pred = reducer_fn(preds)
  
    predictions.append({"id":int(v_id), "y":pred})
    
  return pd.DataFrame(predictions).sort_values("id")

    
def avg_reducer_fn(pred_lst):
  return sum(pred_lst)[0]/len(pred_lst)

In [34]:
#@title Loading data (videos) into numpy array
dir_path = os.getcwd()

train_folder = os.path.join(dir_path,"data/train/")
test_folder = os.path.join(dir_path,"data/test/")
train_target = os.path.join(dir_path,'data/train_target.csv')

print("Current dir -> ", dir_path)
print("Train folder -> ",train_folder)
print("Train target -> ",train_target)
print("Test folder -> ",test_folder)

X_train = get_videos_from_folder(train_folder)
y_train = get_target_from_csv(train_target)
X_test = get_videos_from_folder(test_folder)

print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}")


Current dir ->  /content/drive/aml-task4
Train folder ->  /content/drive/aml-task4/data/train/
Train target ->  /content/drive/aml-task4/data/train_target.csv
Test folder ->  /content/drive/aml-task4/data/test/
X_train: (158,)
y_train: (158,)
X_test: (69,)


In [0]:
#@title Splitting data into training, validation and test set and converting them to frame based approach


random_state = 42


print(f"Splitting Data into Training and Validation Set: ")
X_train_split, X_valid_split, y_train_split, y_valid_split = train_test_split(X_train, y_train, test_size=16, random_state=random_state)


print(f"X_train_split: {X_train_split.shape}")
print(f"y_train_split: {y_train_split.shape}")
print(f"X_valid_split: {X_valid_split.shape}")
print(f"y_valid_split: {y_valid_split.shape}")

print(f"\nTraining Set Video: Class 0: {y_train_split.shape[0] - sum(y_train_split)} Class 1: {sum(y_train_split)}")
print(f"Validation Set Video: Class 0: {y_valid_split.shape[0] - sum(y_valid_split)} Class 1: {sum(y_valid_split)}")


print(f"\n\nConverting Training and Validation Set into shuffled Frame Based Sets: ")
X_train_frame, y_train_frame, _ = to_frame_based(X=X_train_split, y=y_train_split, random_state=random_state)
X_valid_frame, y_valid_frame, idx_valid_frame = to_frame_based(X=X_valid_split, y=y_valid_split, random_state=random_state)


print(f"X_train_frame: {X_train_frame.shape}")
print(f"y_train_frame: {y_train_frame.shape}")

print(f"\nX_valid_frame: {X_valid_frame.shape}")
print(f"y_valid_frame: {y_valid_frame.shape}")
print(f"idx_valid_frame: {idx_valid_frame.shape}")


print(f"\nTraining Set Frame: Class 0: {y_train_frame.shape[0] - sum(y_train_frame)} Class 1: {sum(y_train_frame)}")
print(f"Validation Set Frame: Class 0: {y_valid_frame.shape[0] - sum(y_valid_frame)} Class 1: {sum(y_valid_frame)}")



print("\n\nTest Set Frame: ")
X_test_frame, idx_test_frame  = to_frame_based(X=X_test, random_state=random_state)

print(f"X_test_frame: {X_test_frame.shape}")
print(f"idx_test_frame: {idx_test_frame.shape}")

In [0]:
# Don't think this is working as expected

#from https://stackoverflow.com/a/46844409
# define roc_callback, inspired by https://github.com/keras-team/keras/issues/6050#issuecomment-329996505
def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.metrics.auc(labels=y_true, predictions=y_pred, num_thresholds=200, curve='ROC', summation_method='trapezoidal')

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value

In [0]:
best_weights_filepath = './best_weights.hdf5'

print("ATTENTION: Roc Auc Metric not correctly working (I think it somehow would need to be reset for validation and new epoch)")


X_train_frame = X_train_frame.reshape(-1,100,100,1)
X_valid_frame = X_valid_frame.reshape(-1,100,100,1)

#create model
model = Sequential()

model.add(Conv2D(40, kernel_size=5, padding="same",input_shape=(100, 100, 1), activation = 'relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(70, kernel_size=3, padding="same", activation = 'relu'))
model.add(Conv2D(500, kernel_size=3, padding="same", activation = 'relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(1024, kernel_size=3, padding="valid", activation = 'relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

adam = optimizers.Adam(lr=0.0001)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy', auc_roc])

saveBestModel = ModelCheckpoint(best_weights_filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
early_stopping = EarlyStopping(monitor='val_acc', patience=0, verbose=1, mode='max')
callbacks = [early_stopping, saveBestModel]

history = model.fit(x=X_train_frame, y=y_train_frame, batch_size=32,
          validation_split=0.0, validation_data=(X_valid_frame, y_valid_frame), # use the explicitly defined validation set
          initial_epoch=0, epochs=5, shuffle=True , steps_per_epoch=None, validation_steps=None, # do epochs number of passes through the complete training dataset shuffling it each time
          class_weight=None, sample_weight=None,
          verbose=1, callbacks=callbacks)

#reload best weights
model.load_weights(best_weights_filepath)


In [30]:
print("Test that the roc auc metric used in keras is correct")

scores = model.evaluate(x=X_valid_frame, y=y_valid_frame, batch_size=32, verbose=1, sample_weight=None, steps=None)
print(f"Scores: {scores}")

y_valid_frame_pred = model.predict(x=X_valid_frame, batch_size=32, verbose=0, steps=None)

roc_auc = roc_auc_score(y_valid_frame, y_valid_frame_pred)
print(f"Roc Auc: {roc_auc}")

Test that the roc auc metric used in keras is correct
943/943 [==============================] - 2s 2ms/step
Scores: [0.4749480928684469, 0.8939554613569508, 0.975812877918478]
Roc Auc: 0.9527256767994088


In [31]:
df = combine_frame_based_pred(y_valid_frame_pred, idx_valid_frame, avg_reducer_fn)
display(df)

roc_auc = roc_auc_score(y_valid_split, df["y"].values)
print(f"Roc Auc of Videos: {roc_auc}")

,id,y
3,0,0.349912
0,1,0.999995
14,2,0.853118
1,3,0.268156
4,4,0.000236
6,5,0.999538
12,6,0.015090
15,7,0.999996
7,8,0.999685
10,9,0.966540


Roc Auc of Videos: 0.9375


In [0]:
submission_name = "nk_frame_based_cnn_more_dropout_earlystopping_loadbestweights.csv"

X_test_frame = X_test_frame.reshape(-1,100,100,1)
y_test_frame_pred = model.predict(x=X_test_frame, batch_size=32, verbose=0, steps=None)

df = combine_frame_based_pred(y_test_frame_pred, idx_test_frame, avg_reducer_fn)
df["y"] = df["y"].round(2)
#display(df)

submission_folder = os.path.join(dir_path,"submissions/")
csv_file = submission_folder + submission_name

with open(csv_file, 'w') as csv:
  df.to_csv(csv,index = False)